In [1]:
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .

Cloning into 'mergekit'...
remote: Enumerating objects: 1287, done.
remote: Counting objects: 100% (702/702), done.
remote: Compressing objects: 100% (285/285), done.
remote: Total 1287 (delta 566), reused 463 (delta 416), pack-reused 585
Receiving objects: 100% (1287/1287), 365.97 KiB | 1.92 MiB/s, done.
Resolving deltas: 100% (869/869), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.2 MB/s eta 0:00

In [2]:
import yaml

MODEL_NAME = "mistral-7b-instruct-code-ties"
yaml_config = """
models:
  - model: unsloth/mistral-7b-instruct-v0.2-bnb-4bit
  - model: akameswa/mistral-7b-instruct-javascript-4bit
    parameters:
      density: 0.85
      weight: 0.25
  - model: akameswa/mistral-7b-instruct-python-4bit
    parameters:
      density: 0.85
      weight: 0.25
  - model: akameswa/mistral-7b-instruct-java-4bit
    parameters:
      density: 0.85
      weight: 0.25
  - model: akameswa/mistral-7b-instruct-javascript-4bit
    parameters:
      density: 0.85
      weight: 0.25
merge_method: ties
base_model: unsloth/mistral-7b-instruct-v0.2-bnb-4bit
parameters:
  normalize: true
dtype: float16
"""

with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)

In [ ]:
!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle

In [5]:
!pip install -qU huggingface_hub

from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template

username = "akameswa"

template_text = """
---
license: apache-2.0
tags:
- merge
- mergekit
- lazymergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## 🧩 Configuration

```yaml
{{- yaml_config -}}
```
"""

jinja_template = Template(template_text.strip())

data = yaml.safe_load(yaml_config)
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")

content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

card = ModelCard(content)
card.save('merge/README.md')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 7.2 MB/s eta 0:00:00


In [7]:
from google.colab import userdata
from huggingface_hub import HfApi

username = "akameswa"

api = HfApi(token="")

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)

model-00002-of-00004.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.57G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/1.96G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/akameswa/mistral-7b-instruct-code-ties/commit/553e9e857fc93828a47f15b4c7581d9506acdceb', commit_message='Upload folder using huggingface_hub', commit_description='', oid='553e9e857fc93828a47f15b4c7581d9506acdceb', pr_url=None, pr_revision=None, pr_num=None)